In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
from IPython.display import display, clear_output
from matplotlib import pyplot as plt
from ultralytics import YOLO
from datetime import datetime
import os

# Load YOLOv8 model
yolo = YOLO('yolov8n.pt')

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Folder for alerts
if not os.path.exists("alerts"):
    os.makedirs("alerts")

def save_violation(frame, reason="violation"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"alerts/{reason}_{timestamp}.jpg"
    cv2.imwrite(filename, frame)
    print(f"📸 Saved screenshot: {filename}")

def detect_objects(frame):
    return yolo.predict(frame, imgsz=640, verbose=False)[0]

def detect_multiple_faces(results):
    return len(results.multi_face_landmarks) if results.multi_face_landmarks else 0

def draw_alert(frame, text, color=(0, 0, 255)):
    cv2.putText(frame, text, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 3)

def show_frame_in_notebook(frame):
    """Display frame inside Jupyter Notebook"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame_rgb)
    plt.axis('off')
    display(plt.gcf())
    clear_output(wait=True)

def main():
    violation_count = 0
    MAX_WARNINGS = 3
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("❌ Webcam access failed")
        return

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_results = face_mesh.process(frame_rgb)
            object_results = detect_objects(frame)

            # Multiple faces
            num_faces = detect_multiple_faces(face_results)
            if num_faces > 1:
                draw_alert(frame, "⚠️ MULTIPLE FACES DETECTED")
                save_violation(frame, "multiple_faces")
                violation_count += 1

            # Prohibited objects
            for box in object_results.boxes:
                cls = int(box.cls[0])
                label = object_results.names[cls]
                if label.lower() in ['cell phone', 'book', 'laptop', 'remote']:
                    draw_alert(frame, f"⚠️ {label.upper()} DETECTED")
                    save_violation(frame, "prohibited_object")
                    violation_count += 1
                    break

            # Draw violation count
            cv2.putText(frame, f"Violations: {violation_count}/{MAX_WARNINGS}", (30, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

            # Stop after 3 warnings
            if violation_count >= MAX_WARNINGS:
                draw_alert(frame, "🛑 EXAM TERMINATED")
                save_violation(frame, "exam_terminated")
                show_frame_in_notebook(frame)
                break

            # Show frame in notebook
            show_frame_in_notebook(frame)

            time.sleep(0.05)

    finally:
        cap.release()
        print("✅ Session Ended")

main()


📸 Saved screenshot: alerts/prohibited_object_20250704_180324.jpg
📸 Saved screenshot: alerts/exam_terminated_20250704_180324.jpg
